In [1]:
using LinearAlgebra

### Scalar Neural Network

In [2]:
h(x) = exp(-x)
h′(x,y) = -y
𝓁(x,y) = sum(abs2,x-y)/2
𝓁′(x,y) = x-y
𝜀 = .0001

0.0001

In [3]:
N = 3 
w = randn(N)
b = randn(N)
params =[[w[i],b[i]] for i=1:N]

3-element Array{Array{Float64,1},1}:
 [0.81055, -0.372367]
 [2.74526, 1.16666]  
 [-0.219741, 0.53361]

In [4]:
function neural_net(params, input; h=h, h′=h′, N=length(params))
    δ = [];
    X = [input];
    for i=1:N
        x = params[i]⋅[X[i],1]
        y = h.(x) 
        push!(δ, h′.(x,y))
        push!(X,y)
    end
    return X,δ
end

neural_net (generic function with 1 method)

In [5]:
x = rand()
y = 0.5
X,δ   = neural_net(params,x)
L     = Bidiagonal(zeros(N),δ[2:N].*w[2:N],:L)
D     = Diagonal(δ.*[[X[i],1]' for i=1:N])
f     = [zeros(N-1);𝓁′(X[N+1],y)]
∇J    = D'*((I-L')\f)

3-element Array{Array{Float64,1},1}:
 [0.000200208, 0.00054681]   
 [-0.000199184, -0.000184682]
 [-0.00084045, -0.0521265]   

In [6]:
∇Jfd = ∇J * 0
ϵ    = ∇J * 0
for i=1:N, j=1:2       
    ϵ[i][j] = 𝜀
    ∇Jfd[i][j]=(𝓁(neural_net(params.+ϵ,x)[1][N+1],y)-𝓁(neural_net(params.-ϵ,x)[1][N+1],y))/2𝜀
    ϵ[i][j] = .0
end
∇Jfd

3-element Array{Array{Float64,1},1}:
 [0.000200208, 0.00054681]   
 [-0.000199184, -0.000184682]
 [-0.00084045, -0.0521265]   

### Simple Matrix Neural Network
- [] needed to create a box type
- [] There are some issues with adjoint, did we assume transpose recursive in the article? If it is below code works accordingly.
- [] However, adjoint definitions for ⊗, ⊗′ problematic.

In [7]:
import Base: +,-,*,/,zero,one,adjoint,convert,inv,size,iszero

abstract type Op; end
struct (⊗) <: Op; A; B; end
struct ⊗′ <: Op; A; B; end
struct Δ  <: Op; A; end

-(K::⊗′) = -K.A ⊗′ K.B 
*(K::⊗′,X::Union{AbstractArray,Number}) = K.B * (K.A * X) # changing paranthesis gives wrong result
adjoint(K::⊗′) = K.B ⊗′ K.A' # same issue with kronocker adjoint

-(K::⊗) = -K.A ⊗ K.B
*(K::⊗,X::Union{AbstractArray,Number}) = (K.B * X) * K.A' # changing paranthesis gives dimension errors
adjoint(K::⊗) = K.A' ⊗ K.B  # not consistent with adjoint of kroncker, if we keep consistent
                            # then the elements of D becomes X[j]' ⊗ Δ(δ[i])', not consistent with article

-(X::Δ) = Δ(-X.A)
*(X::Δ,Y::Union{AbstractArray,Number}) = X.A .* Y
*(Y::Union{AbstractArray,Number},X::Δ) = Y .* X.A
adjoint(X::Δ) = Δ(X.A')

# I can't think another way than boxing elements 
#to handle with zeros and ones requires by the backslash and triangular matrices
struct Box; K; end
# Unary Definitions
zero(::Type{Box}) = Box(0)
one(::Type{Box}) = Box(1)
zero(::Box) = zero(Box)
one(::Box) = one(Box)
value(R::Box) = R.K
iszero(R::Box) = R.K==0
adjoint(R::Box) = Box(adjoint(R.K))
inv(R::Box) = Box(inv(R.K))
#Binary Definitions
convert(::Type{Box},x::Union{Number,V,T}) where V <: Op where T <: AbstractArray = Box(x)
/(X::Number,R::Box) = Box(X*inv(R))
/(R1::Box,R2::Box) = Box(R1*inv(R2))
-(R::Box) = Box(-R.K)
-(R::Box, X::AbstractArray) = Box(R.K-X)
-(X::AbstractArray,R::Box)  = Box(X-R.K)
-(R1::Box,R2::Box) = Box(R1.K.-R2.K)
+(R1::Box,R2::Box) = Box(R1.K.+R2.K)
*(R1::Box,R2::Box) = Box(R1.K*R2.K)
*(R::Box,X::Union{Number,V,T}) where V <: Op where T <: AbstractArray = Box(R.K * X)
*(X::Union{Number,V,T},R::Box) where V <: Op where T <: AbstractArray = Box(X * R.K)

* (generic function with 350 methods)

In [8]:
#Needed to overwrite naivesub!, see comment in 12th line
import LinearAlgebra: naivesub!, has_offset_axes
function naivesub!(A::UnitUpperTriangular, b::AbstractVector, x::AbstractVector = b)
    @assert !has_offset_axes(A, b, x)
    n = size(A, 2)
    if !(n == length(b) == length(x))
        throw(DimensionMismatch("second dimension of left hand side A, $n, length of output x, $(length(x)), and length of right hand side b, $(length(b)), must be equal"))
    end
    @inbounds for j in n:-1:1
        xj = x[j] = b[j]
        for i in j-1:-1:1
            if !iszero(A.data[i,j]) # EKIN: added this line to solve zero shape related problem. 
                                    # Otherwise, in matrix neural network we get dimension mismastch
                b[i] -= A.data[i,j] * xj
            end
        end
    end
    x
end

array(x) = fill(x,1,1)

array (generic function with 1 method)

In [9]:
function neural_net(params,input;h=h,h′= h′)
    X     = [input]
    δ     = []
    for i=1:length(params)
        x = params[i][1]*X[i] .+ params[i][2]         
        push!(X,h.(x))
        push!(δ,h′.(x,X[i+1]))
    end 
    X,δ
end

neural_net (generic function with 1 method)

In [10]:
n = [3 3 3 1]
N = length(n)-1
B = 7
W = [randn(n[i+1],n[i]) for i=1:N]
b = [randn(n[i+1]) for i=1:N]
params =[[W[i],b[i]] for i=1:N]

3-element Array{Array{Array{Float64,N} where N,1},1}:
 [[-0.049471 0.396971 0.706293; 0.298809 0.394277 -0.539425; -0.319241 0.927687 -0.20654], [-0.842948, 0.030618, 0.674276]]
 [[-0.129688 0.472665 0.465445; -0.395151 1.22697 -1.40906; 0.481917 0.778302 0.200718], [0.708873, -1.27867, -0.579021]]  
 [[-2.06629 0.817443 0.960496], [-1.60267]]                                                                                

In [11]:
y = randn(1,B)*0.1
x = randn(n[1],B)*0.1
X,δ = neural_net(params,x)
D = Diagonal([[X[i]' ⊗  Δ(δ[i]) Δ(δ[i]')] for i=1:N])
L = Bidiagonal(fill(Box(0),N), [Box(params[i][1] ⊗′ Δ(δ[i])) for i=2:N] , :L)
f = [Box(0) for i=1:N]
f[N] = 𝓁′(X[N+1],y)
∇J = D'*array.((UnitUpperTriangular(-L')\f))

3-element Array{Array{Any,2},1}:
 [Box([-0.00928111 -0.0082192 0.0257615; 0.0166787 0.0150493 -0.0442492; -0.00925947 -0.00969559 0.0275945]); Box([0.00784546 0.0479041 … 0.101229 0.0590256; -0.0119298 -0.072742 … -0.180598 -0.0828348; 0.00852024 0.0476893 … 0.108329 0.0513433])]
 [Box([-0.137948 -0.0650087 -0.0329402; 0.878694 0.413408 0.210159; 0.0505024 0.0238361 0.0120744]); Box([-0.00143202 -0.00807506 … -0.0224207 -0.0120017; 0.0102811 0.057048 … 0.136611 0.0752457; 0.000552706 0.00267994 … 0.0083395 0.0043266])]    
 [Box([-0.0302885 -0.487081 -0.0238861]); Box([-0.00218138 -0.0125983 … -0.0361146 -0.0168697])]                                                                                                                                                       

In [12]:
∇Jfd = params*0
ϵ=params*0
for i=1:length(params), wb=1:2
    for j=1:length(ϵ[i][wb])
            ϵ[i][wb][j] = 𝜀
            ∇Jfd[i][wb][j] =(𝓁(neural_net(params+ϵ,x)[1][N+1],y)-𝓁(neural_net(params-ϵ,x)[1][N+1],y))/2𝜀
            ϵ[i][wb][j] = .0
     end
end
∇Jfd

3-element Array{Array{Array{Float64,N} where N,1},1}:
 [[-0.00928111 -0.0082192 0.0257615; 0.0166787 0.0150493 -0.0442492; -0.00925947 -0.00969559 0.0275945], [0.304989, -0.495064, 0.309034]]
 [[-0.137948 -0.0650087 -0.0329402; 0.878695 0.413408 0.210159; 0.0505024 0.0238361 0.0120744], [-0.0625849, 0.398161, 0.0229425]]       
 [[-0.0302885 -0.487082 -0.0238861], [-0.0966351]]                                                                                       

In [13]:
∇Jfd[1][1]

3×3 Array{Float64,2}:
 -0.00928111  -0.0082192    0.0257615
  0.0166787    0.0150493   -0.0442492
 -0.00925947  -0.00969559   0.0275945

In [14]:
value(∇J[1][1])

3×3 Array{Float64,2}:
 -0.00928111  -0.0082192    0.0257615
  0.0166787    0.0150493   -0.0442492
 -0.00925947  -0.00969559   0.0275945

In [15]:
∇Jfd[1][2]

3-element Array{Float64,1}:
  0.30498881260977717
 -0.49506397154910897
  0.30903439150652134

In [16]:
sum(value(∇J[1][2]);dims=2) # biasses needs to be sum up in batch dimension, since they are broadcasted.

3×1 Array{Float64,2}:
  0.3049888100342567 
 -0.4950639037319157 
  0.30903438608774825

### Densely Connected Matrix Network

 - [ ] Defined step matrices to make D'* possible with generic zeros.

In [17]:
import Base: getindex, setindex!
"""
Horizontal Step Matrix
    [---
        ---
           ---]

Vertical Step Matrix
    [|
     |
       |
       |
       |
         |]
"""
abstract type StepMatrix{T} <: AbstractMatrix{T}; end;
size(S::StepMatrix) = (S.m, S.n)

struct VerticalStepMatrix{T} <: StepMatrix{T}
    m::Int
    n::Int
    colptr::Vector{Int}
    values::Vector{T}    
    function VerticalStepMatrix{T}(m::Int, n::Int, colptr::Vector{Int},values::Vector{T}) where T
        m < 0 && error("rows m $m")
        n < 0 && error("rows n $n")
        new(Int(m), Int(n), colptr, values)
    end
end

getindex(S::VerticalStepMatrix{T},i::Integer, j::Integer) where T = (S.colptr[i] == j ? values[i] : zero(T))
setindex!(S::VerticalStepMatrix{T},value::T,i::Integer,j::Integer) where T = (S.colptr[i] = j; values[i] = value)
VerticalStepMatrix(t::Type{T},m::Int,n::Int) where T  = VerticalStepMatrix{T}(m, n, Vector{Int}(undef,m),Vector{T}(undef,m))

struct HorizontalStepMatrix{T} <: StepMatrix{T}
    m::Int
    n::Int
    rowptr::Vector{Int}
    values::Vector{T}    
    function HorizontalStepMatrix{T}(m::Int, n::Int, colptr::Vector{Int},values::Vector{T}) where T
        m < 0 && error("rows m $m")
        n < 0 && error("rows n $n")
        new(Int(m), Int(n), colptr, values)
    end
end
getindex(S::HorizontalStepMatrix{T},i::Integer, j::Integer) where T = (S.rowptr[j] == i ? S.values[j] : zero(T))
setindex!(S::HorizontalStepMatrix{T},value::T, i::Integer, j::Integer) where T = (S.rowptr[j] = i; S.values[j] = value)
HorizontalStepMatrix(t::Type{T},m::Integer, n::Integer) where T  = HorizontalStepMatrix{T}(m, n, Vector{Int}(undef,n),Vector{T}(undef,n))

adjoint(S::VerticalStepMatrix{T}) where T = HorizontalStepMatrix{T}(S.n,S.m, S.colptr, adjoint.(S.values))
adjoint(S::HorizontalStepMatrix{T}) where T = VerticalStepMatrix{T}(S.n,S.m, S.rowptr, adjoint.(S.values))

function (*)(S::VerticalStepMatrix{T}, X::AbstractVector) where T
    results = Vector{T}(undef,S.m)
    for i=1:S.m
        results[i] = S.values[i] * X[S.colptr[i]]
    end
    return results
end

* (generic function with 351 methods)

In [18]:
function neural_net(params,input;h=h,h′= h′)
    X     = [input]
    δ     = []
    layeroffset = 0; i = 1
    while layeroffset < length(params)
       x = zeros(n[i+1],B)
       for j=1:i
           x += params[layeroffset+j]*X[j]  
       end    
       push!(X,h.(x .+ params[layeroffset+i+1]))
       push!(δ,h′.(x,X[i+1]))
       i+=1; layeroffset+=i
    end 
    X,δ
end

neural_net (generic function with 1 method)

In [19]:
n = [2 3 2 1]
N = length(n)-1
B = 7
params = []
for i=1:N
    wi = []
    for j=1:i
        push!(params,randn(n[i+1],n[j]))
    end
    push!(params,randn(n[i+1]))
end

In [20]:
y = randn(1,B)*0.1
x = randn(n[1],B)*0.1
X,δ = neural_net(params,x)
D = HorizontalStepMatrix(Box,N,sum(2:N+1))
L = LowerTriangular(zeros(Box,N,N))
for i=1:N
    layeroffset = sum(2:i)
    for j=1:i
        D[i,layeroffset+j] = Box(X[j]' ⊗ Δ(δ[i]))
        if i>1 && j!=i
            L[i,j] = Box(params[layeroffset+j+1] ⊗′ Δ(δ[i]))
        end
    end
    D[i,layeroffset+i+1] = Box(Δ(δ[i]'))
end

f = [Box(0) for i=1:N] # [zeros(n[i+1],B) for i=1:N] may use this without modifying naivesub!. However,matrix neural network won't work 
f[N] = 𝓁′(X[N+1],y)

∇J=D'*(UnitUpperTriangular(-L')\f) # Trick: I-L' = UnitUpperTriangular(-L')
∇J=value.(∇J)

9-element Array{Array{Float64,2},1}:
 [-1.36359e-5 9.07862e-5; 4.37527e-6 1.543e-5; -7.8821e-6 4.56613e-5]                                                                                       
 [0.000594278 -5.12599e-5 … -0.000350615 0.00174952; 0.000141013 -9.53719e-6 … -0.000137171 0.000485491; 0.000280163 -2.47653e-5 … -0.000144873 0.000786784]
 [-3.20617e-7 4.22191e-6; -1.35161e-6 6.97848e-5]                                                                                                           
 [-2.35714e-5 -1.62807e-5 -1.46287e-5; -0.000295201 -0.000201111 -0.000187431]                                                                              
 [2.07296e-5 -1.12619e-6 … -9.49229e-6 5.54215e-5; 0.000526085 -4.14134e-5 … -0.000395982 0.0016692]                                                        
 [-1.68009e-6 -2.14735e-5]                                                                                                                                  
 [0.000104647 7.61552

In [21]:
∇Jfd = params*0
ϵ=params*0
for i=1:length(params)
    for j=1:length(ϵ[i])
            ϵ[i][j] = 𝜀
            ∇Jfd[i][j] =(𝓁(neural_net(params+ϵ,x)[1][N+1],y)-𝓁(neural_net(params-ϵ,x)[1][N+1],y))/2𝜀
            ϵ[i][j] = .0
     end
end
∇Jfd

9-element Array{Array{Float64,N} where N,1}:
 [-1.36359e-5 9.07862e-5; 4.37527e-6 1.543e-5; -7.8821e-6 4.56613e-5]         
 [-0.000335133, -6.85026e-5, -0.000180574]                                    
 [-3.20617e-7 4.22191e-6; -1.35161e-6 6.97848e-5]                             
 [-2.35714e-5 -1.62807e-5 -1.46287e-5; -0.000295201 -0.000201111 -0.000187431]
 [-2.11552e-5, -0.000260842]                                                  
 [-1.68009e-6 -2.14735e-5]                                                    
 [0.000104647 7.61552e-5 6.55572e-5]                                          
 [0.000104149 0.000469868]                                                    
 [9.5503e-5]                                                                  

In [22]:
∇J[1]

3×2 Array{Float64,2}:
 -1.36359e-5  9.07862e-5
  4.37527e-6  1.543e-5  
 -7.8821e-6   4.56613e-5

In [23]:
∇Jfd[1]

3×2 Array{Float64,2}:
 -1.36359e-5  9.07862e-5
  4.37527e-6  1.543e-5  
 -7.8821e-6   4.56613e-5

In [24]:
∇J[2] # biasses needs to be sum up in batch dimension, since they are broadcasted.

3×7 Array{Float64,2}:
 0.000594278  -5.12599e-5  -0.000559212  …  -0.000350615  0.00174952 
 0.000141013  -9.53719e-6  -0.000137091     -0.000137171  0.000485491
 0.000280163  -2.47653e-5  -0.000257162     -0.000144873  0.000786784

In [25]:
sum(∇J[2];dims=2)

3×1 Array{Float64,2}:
 -0.00033513258877194453
 -6.850257717373547e-5  
 -0.00018057410778917003

In [26]:
∇Jfd[2]

3-element Array{Float64,1}:
 -0.00033513258237971577
 -6.850257718710173e-5  
 -0.00018057410691718623